## Hartree Fock 

FC = eSC

F = T + V_ne + V_ee                         
->Fock matrix acts on the splitted primitive basis functions ( Gaussian functions) 

S = overlap matrix
C matrix is the coeffient matrix, the rows contain the atomic slater bases


for H2 and then H2O

T = Kinetic energy matrix, where number of rows and number of column are equal to total number of slater functions

Note: atomic orbital are described with the slater functions, which themselves are L.C. of gaussian functions
and interestingly the atomic orbitals also might be described with multiple slater fucntions: 6-31G

In [5]:
import sys
import numpy as np

In [6]:
class primitive_gaussian():
    
    def __init__(self, alpha, coeff, coordinates, l1, l2, l3):
    
        self.alpha = alpha
        self.coeff = coeff
        self. coordinates = np.array(coordinates)
        self.l1 = l1
        self.l2 = l2
        self.l3 = l3
        # Normalisation constant
        self.A = (2.0* alpha / np.pi) ** 0.75  # + other terms when we have l1, l2 ,l3 (for p orbital, d orbital)
        

In [18]:
def kinetic(molecule):
    nbasis = len(molecule)
    
    T = np.zeros([nbasis, nbasis])

   
    # looping over the T matrix, whose rows and columns are equal to number of atomic slater fucntions
    
    for i in range(nbasis):            # row of matrix
        for j in range(nbasis):        # column of matrix
            
            # taking account of all the primitive(gaussian fucntions) bases present in the atomic slater function  
            
            nprimitives_i = len(molecule[i])   # number of gaussian describing the ith slater fucntion
            nprimitives_j = len(molecule[j])    # number of gaussian describing the jth slater function
            
            for k in range(nprimitives_i):
                for l in range(nprimitives_j):
                    
                    c1c2 = molecule[i][k].coeff * molecule[j][l].coeff
                    N = molecule[i][k].A * molecule[j][l].A
                    p = molecule[i][k].alpha + molecule[j][l].alpha
                    q = molecule[i][k].alpha * molecule[j][l].alpha / p
                    Q = molecule[i][k].coordinates - molecule[j][l].coordinates
                    Q2 = np.dot(Q, Q)
                    
                    P = molecule[i][k].alpha * molecule[i][k].coordinates + molecule[j][l].alpha * molecule[j][l].coordinates
                    Pp = P / p
                    PG = Pp - molecule[j][l].coordinates
                    PGx_2 = PG[0] * PG[0]
                    PGy_2 = PG[1] * PG[1]
                    PGz_2 = PG[2] * PG[2]
                    
                    
                    s = N * c1c2* np.exp(-q*Q2) * (np.pi/p)**(3/2)
                    
                    T[i,j] += 3 * molecule[j][l].alpha*s
                    T[i,j] -= 2 * molecule[j][l].alpha*molecule[j][l].alpha *s * (PGx_2 + 0.5/p)
                    T[i,j] -= 2 * molecule[j][l].alpha*molecule[j][l].alpha *s * (PGy_2 + 0.5/p)
                    T[i,j] -= 2 * molecule[j][l].alpha*molecule[j][l].alpha *s * (PGz_2 + 0.5/p)
                    
    return T

In [19]:
# STO-3G basis for 1s orbital on hydrogen
# These are descriptions of atomic orbital basis 
                            # Alpha,           # coeff            # Coord   # angular momentum
H1_pg1a = primitive_gaussian (0.3425250914E+01, 0.1543289673E+00, [0,0,0], 0, 0, 0)
H1_pg1b = primitive_gaussian (0.6239137298E+00 , 0.5353281423E+00, [0,0,0], 0, 0, 0)
H1_pg1c = primitive_gaussian (0.1688554040E+00, 0.4446345422E+00, [0,0,0], 0, 0, 0)

H2_pg1a = primitive_gaussian (0.3425250914E+01, 0.1543289673E+00, [1.4,0,0], 0, 0, 0)
H2_pg1b = primitive_gaussian (0.6239137298E+00 , 0.5353281423E+00, [1.4,0,0], 0, 0, 0)
H2_pg1c = primitive_gaussian (0.1688554040E+00, 0.4446345422E+00, [1.4,0,0], 0, 0, 0)

H1_1s = [H1_pg1a, H1_pg1b, H1_pg1c]
H2_1s = [H2_pg1a, H2_pg1b, H2_pg1c]

molecule = [H1_1s, H2_1s]

# type(molecule)
# len(molecule[0])

T_integral = kinetic(molecule)
print(T_integral)

[[0.76003188 0.23645466]
 [0.23645466 0.76003188]]


In [20]:
# 6-31G basis for 1s and 2s orbitals on hydrogen

H1_pg1a = primitive_gaussian (0.1873113696E+02, 0.3349460434E-01, [0,0,0], 0, 0, 0)
H1_pg1b = primitive_gaussian (0.2825394365E+01  , 0.2347269535E+00, [0,0,0], 0, 0, 0)
H1_pg1c = primitive_gaussian (0.6401216923E+00, 0.8137573261E+00, [0,0,0], 0, 0, 0)
H1_pg2a = primitive_gaussian (0.1612777588E+00, 1.0000000, [0,0,0], 0, 0, 0)

H2_pg1a = primitive_gaussian (0.1873113696E+02, 0.3349460434E-01, [1.4,0,0], 0, 0, 0)
H2_pg1b = primitive_gaussian (0.2825394365E+01  , 0.2347269535E+00, [1.4,0,0], 0, 0, 0)
H2_pg1c = primitive_gaussian (0.6401216923E+00, 0.8137573261E+00, [1.4,0,0], 0, 0, 0)
H2_pg2a = primitive_gaussian (0.1612777588E+00, 1.0000000, [1.4,0,0], 0, 0, 0)

H1_1s = [H1_pg1a, H1_pg1b, H1_pg1c]
H1_2s = [H1_pg2a]
H2_1s = [H2_pg1a, H2_pg1b, H2_pg1c]
H2_2s = [H2_pg2a]


molecule = [H1_1s, H1_2s, H2_1s,  H2_2s]

T_integral = kinetic(molecule)
print(T_integral)

[[1.39567838 0.259735   0.23200115 0.16601725]
 [0.259735   0.24191664 0.16601725 0.18478593]
 [0.23200115 0.16601725 1.39567838 0.259735  ]
 [0.16601725 0.18478593 0.259735   0.24191664]]
